# Model Training and Validation with PyTorch

This notebook demonstrates the training and validation of a ResNet50 model on a custom dataset with and without random erasing augmentation.


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import os
import copy

## Data Preparation

Loading datasets, applying transformations, and setting up data loaders.


In [ ]:
# Data Transforms
data_transforms = {
    'train_with_erasing': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.ToTensor(),
        transforms.RandomErasing(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]),
    'train_without_erasing': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]),
    'val': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]),
}

# Dataset Paths
dataset_paths = {
    'train_with_erasing': './dataset/training_dataset',
    'train_without_erasing': './dataset/training_dataset',
    'val': './dataset/validation_dataset',
}

# Load Datasets
image_datasets = {
    x: torchvision.datasets.ImageFolder(dataset_paths[x], data_transforms[x]) 
    for x in ['train_with_erasing', 'train_without_erasing', 'val']
}

# Data Loaders
dataloaders = {
    x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4, shuffle=True) 
    for x in ['train_with_erasing', 'train_without_erasing', 'val']
}

## Initialize Models

Initializing ResNet50 models for training with and without random erasing.


In [ ]:
# Initialize Models
model_with_erasing = models.resnet50(pretrained=True)
model_with_erasing.fc = nn.Linear(model_with_erasing.fc.in_features, 2)

model_without_erasing = models.resnet50(pretrained=True)
model_without_erasing.fc = nn.Linear(model_without_erasing.fc.in_features, 2)

# Define Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer_with_erasing = optim.SGD(model_with_erasing.parameters(), lr=0.001, momentum=0.9)
optimizer_without_erasing = optim.SGD(model_without_erasing.parameters(), lr=0.001, momentum=0.9)

## Model Training

Defining the training loop and training both models.


In [ ]:
def train_model(model, dataloader, optimizer, device, num_epochs=30):
    best_acc = 0.0
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        running_corrects = 0
        
        for inputs, labels in dataloader['train_with_erasing']:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            _, preds = torch.max(outputs, 1)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
        
        epoch_loss = running_loss / len(dataloader['train_with_erasing'].dataset)
        epoch_acc = running_corrects.double() / len(dataloader['train_with_erasing'].dataset)
        
        print(f'Epoch {epoch+1}/{num_epochs} - Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
        
        if epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())

    model.load_state_dict(best_model_wts)
    return model

def validate_model(model, dataloader,device):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in dataloader['val']:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    accuracy = accuracy_score(all_labels, all_preds)
    return accuracy

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()
torch.cuda.memory_summary(device=None, abbreviated=False)


model_with_erasing = train_model(model_with_erasing.to(device), dataloaders, optimizer_with_erasing, device)
model_without_erasing = train_model(model_without_erasing.to(device), dataloaders, optimizer_without_erasing, device)

## Model Validation

Validating both models and comparing their performance.


In [ ]:
accuracy_with_erasing = validate_model(model_with_erasing, dataloaders['val'])
accuracy_without_erasing = validate_model(model_without_erasing, dataloaders['val'])

## Plotting Comparison

Plotting the validation accuracies of both models for comparison.


In [ ]:
plt.bar(['With Erasing', 'Without Erasing'], [accuracy_with_erasing, accuracy_without_erasing])
plt.xlabel('Model')
plt.ylabel('Validation Accuracy')
plt.title('Model Comparison')
plt.savefig('model_comparison.png')
plt.show()